## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


##  First，import some package

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt


## Second,I'll creat some tools

### general functions

In [3]:

def read_image(filepath):
    return cv2.imread(filepath)

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    # return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


def gaussian_smoothing(gray_img,kernel_size = 5):

    return cv2.GaussianBlur(gray_img,(kernel_size, kernel_size),0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_outside_line(img,vertices,color=[0, 0, 255], thickness=3):
    cv2.line(img,tuple(vertices[0][0]),tuple(vertices[0][1]),[0,255,0],thickness)
    cv2.line(img,tuple(vertices[0][2]),tuple(vertices[0][3]),[0,255,0],thickness)
    cv2.line(img,tuple(vertices[0][1]),tuple(vertices[0][2]),[0,255,0],thickness)
    cv2.line(img,tuple(vertices[0][4]),tuple(vertices[0][5]),[0,255,0],thickness)
    cv2.line(img,tuple(vertices[0][5]),tuple(vertices[0][6]),[0,255,0],thickness)
    
def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

### camera calibration

In [4]:
def get_images(files = '../camera_cal/calibration*.jpg'):
    '''
    获取校准文件数组
    '''
    return glob.glob(files)



def get_points(images,point_size = (9,6)):
    '''
    获取校准点的数组
    '''

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Step through the list and search for chessboard corners
    for fname in images:
        img = read_image(fname)
        gray = grayscale(img)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, point_size,None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)
    return objpoints,imgpoints

def cal_undistort(img, objpoints, imgpoints):
    '''
    获取校准后的图像
    '''

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist,mtx,dist


###  create a thresholded binary image

#### Sobel

In [5]:
def abs_sobel_thresh(gray_img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    '''
    获取某个维度上经过索贝尔算子获得的边缘图
    '''
    
    # Apply the following steps to img
    # 1) Convert to grayscale
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    # 3) Take the absolute value of the derivative or gradient
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    # 5) Create a mask of 1's where the scaled gradient magnitude 
            # is > thresh_min and < thresh_max
    # 6) Return this mask as your binary_output image
    
    if orient == "x":
        sobel = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0,ksize = sobel_kernel)
    elif orient == "y":
        sobel = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1,ksize = sobel_kernel)
    
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    mask = (scaled_sobel > thresh_min) & (scaled_sobel < thresh_max)
    binary_output = np.zeros_like(gray_img)
    binary_output[mask] = 1
    
    return binary_output
    return grad_binary

def mag_thresh(gray_img, sobel_kernel=3, mag_thresh=(0, 255)):
    '''
    获取x轴和y轴合成的梯度大小在范围内的边缘图
    '''
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Calculate the magnitude 
    # 4) Scale to 8-bit (0 - 255) and convert to type = np.uint8
    # 5) Create a binary mask where mag thresholds are met
    # 6) Return this mask as your binary_output image
    
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1,ksize = sobel_kernel)
    sobel = np.sqrt(sobelx ** 2+ sobely ** 2)
    scaled_sobel = np.uint8(255*sobel/np.max(sobel))
    mask = (scaled_sobel > mag_thresh[0]) & (scaled_sobel < mag_thresh[1])
    mag_binary = np.zeros_like(gray_img)
    mag_binary[mask] = 1
    return mag_binary

def dir_threshold(gray_img, sobel_kernel=3, thresh=(0, np.pi/2)):
    '''
    获取梯度角度在一定范围内的边缘图
    '''
    # 1) Convert to grayscale
    # 2) Take the gradient in x and y separately
    # 3) Take the absolute value of the x and y gradients
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    # 5) Create a binary mask where direction thresholds are met
    # 6) Return this mask as your binary_output image
    
    sobelx = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1,ksize = sobel_kernel)
    # sobel = np.sqrt(sobelx ** 2+ sobely ** 2)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    arctan = np.arctan2(abs_sobely, abs_sobelx)
    mask = (arctan > thresh[0]) & (arctan < thresh[1])
    dir_binary = np.zeros_like(gray_img) # Remove this line
    dir_binary[mask] = 1
    return dir_binary

#### Color space

In [13]:
def hls_select(img,channel = "S", thresh=(0, 255)):
    '''
    获取hls空间某个维度的阈值
    '''

    hls_image = cv2.cvtColor(img,cv2.COLOR_BGR2HLS)
    if channel ==  "S":
        mask = (hls_image[:,:,2] > thresh[0]) &  (hls_image[:,:,2] <= thresh[1])
    elif channel ==  "H":
        mask = (hls_image[:,:,0] > thresh[0]) &  (hls_image[:,:,0] <= thresh[1])
    elif channel == "L":
        mask = (hls_image[:,:,1] > thresh[0]) &  (hls_image[:,:,1] <= thresh[1])
    else:
        return None
    binary_output = np.zeros_like(hls_image[:,:,2]) # placeholder line
    binary_output[mask] = 1
    return binary_output


def bgr_select(image,channel = "R",thresh = (0,255)):
    '''
    获取BGR空间某个维度的阈值
    '''
    if channel ==  "R":
        mask = (image[:,:,2] > thresh[0]) &  (image[:,:,2] <= thresh[1])
    elif channel ==  "B":
        mask = (image[:,:,0] > thresh[0]) &  (image[:,:,0] <= thresh[1])
    elif channel == "G":
        mask = (image[:,:,1] > thresh[0]) &  (image[:,:,1] <= thresh[1])
    else:
        return None
    binary_output = np.zeros_like(image[:,:,2]) # placeholder line
    binary_output[mask] = 1
    return binary_output




###  Rectify binary image

In [ ]:
def recity_unwarp(undist_img, src, dist, mtx, dist):
    '''
    将校正后的图像根据src和dist转化为鸟瞰图
    '''


#         src = np.float32([p1,p2,p3,p4])
#         dst = np.float32([[100,100],[250,100],[250,250],[100,250]])
    M = cv2.getPerspectiveTransform(src, dst)
    M_R = cv2.getPerspectiveTransform(dst,src)
    warped = cv2.warpPerspective(undist_img, M, undist_img.shape[::-1], flags=cv2.INTER_LINEAR)
    return warped, M





### Detect lane pixels and fit to find the lane boundary.

In [14]:
def find_lane_pixels(binary_warped,nwindows = 9,margin = 100,minpix = 50):
    '''
    找到所有的有效的左右车道线坐标值
    '''
    # 获取图像下半部分像素值y轴上相加的直方图
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)

    # 找到图像中点，将其分为左右两部分
    midpoint = np.int(histogram.shape[0]//2)
    
    # 返回每一边最大直方图顶点对应的索引，
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # 超参数
    # 分成几个窗口
    # nwindows = 9
    # 窗口的宽度+/- margin
    # margin = 100
    #寻找中心线的最小像素数
    # minpix = 50

    # 求出窗口的高度
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # 返回二值图非0元素的X坐标元组，y坐标元组
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # 当前窗口中心线的位置，在每个窗口中更新
    leftx_current = leftx_base
    rightx_current = rightx_base

    # 用来储存每个窗口的左右车道的非零像素的索引
    left_lane_inds = []
    right_lane_inds = []

    # 对于每个窗口
    for window in range(nwindows):
        #找到y边界
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height

        #找到x边界
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        

        # 画出窗口的边界线
#         cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
#         cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        # 获取窗口区域的值，分别是从y轴和x轴限制
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        # 将获取到的不为0的x的坐标索引储存
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        

        # 如果获取到的像素点数目大于阈值，则设置下个窗口的中点值，中点值为有效坐标的平均值
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))


    # 有效像素列表进行拼接，拼接为np数组，列表为x和y的坐标数组中的索引
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # 从坐标数组中中获取对应的像素索引的x和y的坐标
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty


def fit_poly(img_shape, leftx, lefty, rightx, righty):
    '''
    拟合左右曲线，以Y轴为自变量，X轴为因变量，
    因为车道线相对来说是比较垂直，以X轴为自变量可能不太好拟合，会存在一个x对应几个y，所有以y为自变量
    '''
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    # Generate x and y values for plotting
    # 获取x轴
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    # 获取左右的多项式，fit为系数，ploty为y值,是y为自变量
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty


def search_around_poly(binary_warped,left_fit,right_fit,margin = 100):
    '''
    通过将拟合的曲线左右延伸来获取下一帧图像的优先检测区域，若检测不到，可以再次从传统窗口检测
    
    '''
    # 超参数，设置拟合线左右的边距 margin = 100
    

    # 获取有效像素
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### TO-DO: Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    
    
    return left_fitx, right_fitx, ploty


### Determine the curvature of the lane and vehicle position with respect to center.

In [ ]:
def measure_curvature_pixels(ploty, left_fit, right_fit):
    '''
    y以像素为单位计算曲率
    '''

    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # 通过公式计算曲率
    left_curverad =((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    
    right_curverad = ((1 + (2 * right_fit[0] * y_eval + right_fit[1] ) ** 2)**1.5)/np.absolute(2*right_fit[0])
    
    
    return left_curverad, right_curverad


def measure_curvature_real(binary_warped,ploty, left_fit_cr, right_fit_cr,ym_per_pix = 30/720,xm_per_pix = 3.7/700):
    '''
    y以现实距离计算曲率
    '''
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # 利用公式计算曲率
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    #计算摄像头中心线和车道中心线的距离
    
    cam_middle = binary_warped.shape[0]/2 * xm_per_pix
    
    
    road_left = (left_fit_cr[0] * (y_eval ** 2)) + (left_fit_cr[1] * y_eval) + left_fit_cr[2]
    road_right = (right_fit_cr[0] * (y_eval ** 2)) + (right_fit_cr[1] * y_eval) + right_fit_cr[2]
    road_middle = (road_right+road_left)/2 * xm_per_pix
    
    distance = cam_middle - road_middle
    
    
    return left_curverad, right_curverad,distance

### show

In [ ]:
def get_draw(binary_warped,left_fitx,right_fitx, ploty):
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)

    # 获取窗口左右的边界

    windows_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])  #将x和y组合成2维的数组，转置为（x，y）
    windows_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])  #转置后再反转
    windows = np.hstack((windows_left, windows_right)) #将点数组合并
    
    

    # Draw the lane onto the warped blank image

    cv2.fillPoly(window_img, np.int_([windows]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    
    # Plot the polynomial lines onto the image
    cv2.circle(out_img,windows_left , 1, (0, 0, 255), 4)
    cv2.circle(out_img,windows_right , 1, (0, 0, 255), 4)
    ## End visualization steps ##
    return out_img

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output after processing
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)
